This code is for the client without any attack 

In [1]:
import torch

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from copy import deepcopy

import numpy as np
import matplotlib.pyplot as plt

from create_MNIST_datasets import get_MNIST, plot_samples

In [2]:
import syft as sy

In [3]:
duet = sy.launch_duet(loopback=True)

🎤  🎸  ♪♪♪ Starting Duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently in beta.
♫♫♫ > Use at your own risk.


    > ❤️ Love Duet? Please consider supporting our community!
    > https://github.com/sponsors/OpenMined

♫♫♫ > Punching through firewall to OpenGrid Network Node at:
♫♫♫ > http://ec2-18-218-7-180.us-east-2.compute.amazonaws.com:5000
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... 
♫♫♫ > DONE!

♫♫♫ > STEP 1: Send the following code to your Duet Partner!

import syft as sy
duet = sy.join_duet(loopback=True)

♫♫♫ > Connecting...


/Users/syeda/courses/fl/lib/python3.8/site-packages/aiortc/rtcdtlstransport.py:211: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  _openssl_assert(lib.SSL_CTX_use_certificate(ctx, self._cert._x509) == 1)  # type: ignore
/Users/syeda/courses/fl/lib/python3.8/site-packages/aiortc/rtcdtlstransport.py:186: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  value=certificate_digest(self._cert._x509),  # type: ignore



♫♫♫ > CONNECTED!

tensor(2.2993, grad_fn=<AddBackward0>) Requests: 0   Messages: 3  Request Handlers: 0                                
tensor(2.2964, grad_fn=<AddBackward0>) Requests: 0   Messages: 3  Request Handlers: 0                                


Each client will launch the duet and from the server side the server would join the duet. When a connection is established, then it would show connected in the drop box and a live status which is shown by a star, would animate.

In [4]:
mnist_iid_train_dls = torch.load('client0.pth')

In [5]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.fc = nn.Linear(784, 10)
        
    def forward(self, x):
        x = torch.flatten(x, 1)
        x = self.fc(x)
        x = F.softmax(x)
        return x


model_client = CNN()

In [6]:
x = duet.store[0].get()

By duet.get - the client got the initial model parameters and the would now run the code upto the end.

In [7]:
torch.nn.utils.vector_to_parameters(x,model_client.parameters())

# Model Training

In [8]:
def loss_classifier(predictions,labels):
    
    m = nn.LogSoftmax(dim=1)
    loss = nn.NLLLoss(reduction="mean")
    
    return loss(m(predictions) ,labels.view(-1))

In [9]:
train_data = mnist_iid_train_dls

epochs = 10
lr =0.1
optimizer=optim.SGD(model_client.parameters(),lr=lr) 
for e in range(epochs): 
    total_loss = 0
    for idx, (features,labels) in enumerate(train_data):
        optimizer.zero_grad()
        predictions= model_client(features)
        loss=loss_classifier(predictions,labels)
        total_loss+=loss;
        loss.backward()
        optimizer.step()
    print(total_loss)
   

tensor(2.3034, grad_fn=<AddBackward0>)
tensor(2.3021, grad_fn=<AddBackward0>)


/var/folders/s3/nn8rqkp92c3413g9w6xsc9y00000gn/T/ipykernel_47265/594803556.py:9: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = F.softmax(x)


tensor(2.3007, grad_fn=<AddBackward0>)
tensor(2.2979, grad_fn=<AddBackward0>)
tensor(2.2948, grad_fn=<AddBackward0>)
tensor(2.2932, grad_fn=<AddBackward0>)
tensor(2.2915, grad_fn=<AddBackward0>)
tensor(2.2898, grad_fn=<AddBackward0>)


In [10]:
list(model_client.parameters())

[Parameter containing:
 tensor([[ 0.0264,  0.0143, -0.0260,  ..., -0.0160,  0.0132,  0.0203],
         [ 0.0353,  0.0115,  0.0053,  ..., -0.0323, -0.0274,  0.0146],
         [ 0.0109, -0.0297, -0.0219,  ...,  0.0022,  0.0330,  0.0248],
         ...,
         [-0.0069, -0.0276, -0.0028,  ..., -0.0208, -0.0108, -0.0067],
         [-0.0155, -0.0122, -0.0277,  ..., -0.0130, -0.0083,  0.0113],
         [ 0.0288,  0.0157, -0.0312,  ..., -0.0159, -0.0133,  0.0251]],
        requires_grad=True),
 Parameter containing:
 tensor([-0.0264,  0.0027, -0.0145, -0.0112,  0.0007, -0.0292, -0.0100,  0.0302,
         -0.0216,  0.0206], requires_grad=True)]

In [11]:
parameters = torch.nn.utils.parameters_to_vector(model_client.parameters())

In [12]:
parameters.send(duet)

/Users/syeda/courses/fl/lib/python3.8/site-packages/torch/_tensor.py:1083: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at  /Users/runner/work/pytorch/pytorch/pytorch/build/aten/src/ATen/core/TensorBody.h:482.)
  return self._grad


In [13]:
duet.store.pandas

,ID,Tags,Description,object_type
0,<UID: 03088f0e51b844a984e93c13d91c333b>,[],,<class 'torch.Tensor'>


In [14]:
duet.requests.add_handler(action="accept")

[2022-12-03T13:24:38.333704-0500][CRITICAL][logger]][47265] You do not have permission to .get() Object with ID: <UID: 03088f0e51b844a984e93c13d91c333b>Please submit a request.
[2022-12-03T13:24:38.341099-0500][CRITICAL][logger]][47265] You do not have permission to .get() Object with ID: <UID: 03088f0e51b844a984e93c13d91c333b>Please submit a request.


The client has sent the model update by .send(duet) and has accepted the request from the server to access the updated model.

The client can see any updates or any data sent from the server in the live status. Also in the live status the request updates from the server will also be shown. If there is no updates then Objects: 0  Requests: 0   Messages: 0  Request Handlers: 0. And if there is any update on any of them the number  